# Μπούφαλης Οδυσσεύς Δημήτριος 03118118
# Παπαζαφειρόπουλος Αναστάσιος 03118079

## Fifo


In [26]:



import numpy as np
from collections import deque
import matplotlib.pyplot as plt

def average(list_A):
    return(sum(list_A)/len(list_A))

#for both poisson arrivals and exponential distributed times of service we use the generator -logU/a
#for poissoin arrivals we have a 0.9 arrivals/1000msec -> -logU*(1000/0.9) msec
#for exponential distributed times of service we have 1/a = 54msec for cpu and 1/a = 35msec
def arrivals(average_time,num_points = 1):
    X = - float(np.log(U(num_points))) * average_time
    moment_of_arrivals[average_time].append(X)
    return X

def nextArrival():
    # we have a = 1000/0.9 msec (we use 1000 in order to measure time in mseconds)
    average_time = 1000/0.9
    return arrivals(average_time)

#uniform distribution generator in (0,1)
def U(num_points = 1):
    return float(np.random.uniform(size = num_points))


def next(arrival,cpu,disk):
    if cpu == None:
        if disk != None and disk < arrival:
            return "DISK"
        else:
            return "arrival"
    elif disk == None:
        if cpu != None and cpu < arrival:
            return "CPU"
        else:
            return "arrival"
    else:
        if arrival < disk  and arrival < cpu:
            return "arrival"
        elif cpu < disk:
            return "CPU"
        else:
            return "DISK"



cpu_queue = deque([])
disk_queue = deque([])

cpu_current_job = None
cpu_start_time = None
cpu_remaining_time = None

disk_current_job = None
disk_start_time = None
disk_remaining_time = None

job_id = 1
clock = 0
curr_jobs = 0
old_clock = 0

moment_of_arrivals = {}

for average_time in [1000/0.9, 54, 35]:
    moment_of_arrivals[average_time] = []


arrival_time = {}
departure_time = {}
disk_visits = {}


sum_response_times = []



empty_cpu_times = []
empty_disk_times = []

cycle_lengths   = []
jobs_per_cycle = []


cpu_EMPTY_TIME  = 0
disk_EMPTY_TIME = 0

# cpu arrival time
cpu_arrival = nextArrival()
disk_EMPTY_TIME = cpu_EMPTY_TIME = 0
disk_LAST_EMPTY = cpu_LAST_EMPTY = 0

# cpu processing time
empty_cpu = True


# disk processing time
empty_disk = True
regens = 0


Checkpoint = True
jobs = []

while (Checkpoint and regens < 1000):

    # start of the regen cycle
    # Everything is empty
    if empty_cpu and empty_disk:

        # check the confidence interval every 20 cycles
        if (regens - 1) % 20 == 0 and regens != 1:

            # calculate s ^ 2
            y_bar = average(sum_response_times)
            c_bar = average(jobs_per_cycle)
            n = regens - 1

            sy = 0
            sc = 0
            syc = 0
            for i in range(1,regens):
                sy += (sum_response_times[i] - y_bar) ** 2
                sc += (jobs_per_cycle[i] - c_bar) ** 2
                syc += (sum_response_times[i] - y_bar) * (jobs_per_cycle[i] - c_bar)

            sy /= n - 1
            sc /= n - 1
            syc /= n - 1
            R = average(sum_response_times) / average(jobs_per_cycle)
            s = sy - 2 * R * syc + R ** 2 * sc
            
            # z for 0.95
            z1_a_2 =  1.960
            confidence_interval = z1_a_2 * float(np.sqrt(s)) / (c_bar * float(np.sqrt(n)))

            if confidence_interval / R < 0.1:
                Checkpoint = False
                break

     
        cycle_length = clock-old_clock
        cycle_lengths.append(clock-old_clock)
        old_clock = clock
       
        
        R_cycle = 0
        if jobs:
            for job in jobs:
                R_cycle += (departure_time[job] - arrival_time[job] + 234)
 
            sum_response_times.append(R_cycle)
            jobs_per_cycle.append(len(jobs))
            
        else:
            sum_response_times.append(0)
            jobs_per_cycle.append(0)

        if cycle_length == 0:
            cycle_length = 1

       
        
        empty_cpu_times.append(cpu_EMPTY_TIME)
        cpu_EMPTY_TIME = 0
        

        
        empty_disk_times.append(disk_EMPTY_TIME)
        disk_EMPTY_TIME = 0
   

        regens += 1
        clock = cpu_arrival
        cpu_EMPTY_TIME += (clock - cpu_LAST_EMPTY)

        #initialize jobs again for the new regen cycle
        jobs = []

        # begin the job's execution

        processing_time = arrivals(54)
        cpu_current_job = job_id
        jobs.append(job_id)
        cpu_start_time = clock
        cpu_remaining_time = processing_time

        # find new arrival time 
        arrival_time[job_id] = clock
        disk_visits[job_id] = 0
        cpu_arrival = clock + nextArrival()
        job_id += 1
        curr_jobs += 1
        empty_cpu = False
       

    elif empty_disk:

        # new arrival
        if next(cpu_arrival, cpu_start_time + cpu_remaining_time,None) == "arrival":
            clock = cpu_arrival
            arrival_time[job_id] = clock
            disk_visits[job_id] = 0

            jobs.append(job_id)
            processing_time = arrivals(54)
            cpu_queue.append([job_id,processing_time])
            #find new arrival time
            cpu_arrival = clock + nextArrival()
            empty_cpu = False
            curr_jobs += 1
            job_id += 1
            

        else:
            clock = cpu_start_time + cpu_remaining_time
            # job leaves the system with p = 1/19 
            Uniform = U()
            if (Uniform <= 1/19):
                departure_time[cpu_current_job] = clock
                if cpu_queue:
                    #fifo
                    cpu_current_job, cpu_remaining_time = cpu_queue.popleft()
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job,  cpu_remaining_time, cpu_start_time = None, None,  None
                    cpu_LAST_EMPTY = clock
                    empty_cpu = True
                curr_jobs -= 1
            else:
                # load job to the empty disk
                disk_visits[cpu_current_job] += 1
                processing_time = arrivals(35)
                disk_current_job = cpu_current_job
                disk_remaining_time = processing_time
                disk_start_time = clock
                disk_EMPTY_TIME += (clock - disk_LAST_EMPTY)
                empty_disk = False

                # load new job to the cpu
                if cpu_queue:
                    cpu_current_job, cpu_remaining_time = cpu_queue.popleft()
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job, cpu_remaining_time, cpu_start_time = None,  None, None
                    empty_cpu = True
            


    elif empty_cpu:

        #new arrival
        if next(cpu_arrival, None, disk_start_time + disk_remaining_time) == "arrival":
            clock = cpu_arrival
            arrival_time[job_id] = clock
            disk_visits[job_id] = 0
            jobs.append(job_id)
            cpu_remaining_time = arrivals(54)
            cpu_EMPTY_TIME += (clock - cpu_LAST_EMPTY)
            cpu_current_job = job_id
            cpu_start_time = clock
            cpu_arrival = clock + nextArrival()
            empty_cpu = False
            curr_jobs += 1
            job_id += 1
            

        else:
            clock = disk_start_time + disk_remaining_time
           # load job to the empty cpu
            processing_time = arrivals(54)
            cpu_current_job = disk_current_job 
            cpu_remaining_time = processing_time
            cpu_start_time = clock
            empty_cpu = False

            # load new job to the disk
            if disk_queue:
                disk_current_job, disk_remaining_time = disk_queue.popleft()
                disk_start_time = clock
            elif not disk_queue:
                disk_current_job, disk_remaining_time, disk_start_time = None,None, None
                disk_LAST_EMPTY = clock
                empty_disk = True
          

    else:

        # new arrival
        if next(cpu_arrival, cpu_start_time + cpu_remaining_time, disk_start_time + disk_remaining_time) == "arrival":
            clock = cpu_arrival
            arrival_time[job_id] = clock
            disk_visits[job_id] = 0
            jobs.append(job_id)
            processing_time = arrivals(54)
            cpu_queue.append([job_id,processing_time])
            # define new arrival time and increment number of jobs
            cpu_arrival = clock + nextArrival()
            empty_cpu = False
            curr_jobs += 1
            job_id += 1
            

        elif next(cpu_arrival, cpu_start_time + cpu_remaining_time, disk_start_time + disk_remaining_time) == "CPU":
            clock = cpu_start_time + cpu_remaining_time
            # job leaves the system with pA = 1/10 or pB = 1/8
            Uniform = U()
            if (Uniform <= 1/19):
                departure_time[cpu_current_job] = clock
                if cpu_queue:
                    cpu_current_job, cpu_remaining_time = cpu_queue.popleft()
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job, cpu_remaining_time, cpu_start_time = None, None, None
                    cpu_LAST_EMPTY = clock
                    empty_cpu = True
                curr_jobs -= 1
            else:
                # load job to the disk queue
                disk_visits[cpu_current_job] += 1
                processing_time = arrivals(35)
            
                disk_queue.append([cpu_current_job,processing_time])

                # load new job to the cpu
                if cpu_queue:
                    cpu_current_job, cpu_remaining_time = cpu_queue.popleft()
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job, cpu_remaining_time, cpu_start_time = None,None, None
                    cpu_LAST_EMPTY = clock
                    empty_cpu = True
            

        else:
            clock = disk_start_time + disk_remaining_time
            # load job to the cpu
            processing_time = arrivals(54)
            # Current - A, Coming - A
            cpu_queue.append([disk_current_job,processing_time])


            # load new job to the disk
            if disk_queue:
                disk_current_job, disk_remaining_time = disk_queue.popleft()
                disk_start_time = clock
            elif not disk_queue:
                disk_current_job, disk_remaining_time, disk_start_time = None, None, None
                disk_LAST_EMPTY = clock
                empty_disk = True
            

print("Regen cycles =",regens)



print("Average respone time ",round(float(average(sum_response_times)/average(jobs_per_cycle)),3))
print("cpu utilization",round(float(1 - average(empty_cpu_times)/average(cycle_lengths)),3))
print("Disk utilization",round(float(1 - average(empty_disk_times)/average(cycle_lengths)),3))






Regen cycles = 1000
Average respone time  14929.19
cpu utilization 0.938
Disk utilization 0.56


## Lwtf

In [20]:



import numpy as np
from collections import deque
import matplotlib.pyplot as plt

def average(list_A):
    return(sum(list_A)/(len(list_A)))

#for both poisson arrivals and exponential distributed times of service we use the generator -logU/a
#for poissoin arrivals we have a 0.9 arrivals/1000msec -> -logU*(1000/0.9) msec
#for exponential distributed times of service we have 1/a = 54msec for cpu and 1/a = 35msec
def arrivals(average_time,num_points = 1):
    X = - float(np.log(U(num_points))) * average_time
    moment_of_arrivals[average_time].append(X)
    return X

def nextArrival():
    # we have a = 1000/0.9 msec (we use 1000 in order to measure time in mseconds)
    average_time = 1000/0.9
    return arrivals(average_time)

def U(num_points = 1):
    return float(np.random.uniform(size = num_points))



def next(arrival,cpu,disk):
    if cpu == None:
        if disk != None and disk < arrival:
            return "DISK"
        else:
            return "arrival"
    elif disk == None:
        if cpu != None and cpu < arrival:
            return "CPU"
        else:
            return "arrival"
    else:
        if arrival < disk  and arrival < cpu:
            return "arrival"
        elif cpu < disk:
            return "CPU"
        else:
            return "DISK"







# Initialize
cpu_queue = []
disk_queue = deque([])

cpu_current_job = None
cpu_start_time = None
cpu_remaining_time = None

disk_current_job = None
disk_start_time = None
disk_remaining_time = None

job_id = 1
clock = 0
curr_jobs = 0
old_clock = 0

moment_of_arrivals = {}
for average_time in [1000/0.9, 54, 35]:
    moment_of_arrivals[average_time] = []

# Initialize statistics

arrival_time = {}
departure_time = {}
disk_visits = {}
cpu_delay = {}


sum_response_times = []



empty_cpu_times = []
empty_disk_times = []

cycle_lengths   = []
jobs_per_cycle = []


cpu_EMPTY_TIME  = 0
disk_EMPTY_TIME = 0

# cpu arrival time
cpu_arrival = nextArrival()
disk_EMPTY_TIME = cpu_EMPTY_TIME = 0
disk_LAST_EMPTY = cpu_LAST_EMPTY = 0

# cpu processing time
empty_cpu = True


# disk processing time
empty_disk = True
regens = 0



jobs = []

Checkpoint = True

while (Checkpoint and regens < 1000):
    #print(cpu_queue)
 
    
    # start of the regen cycle
    # Everything is empty
    if empty_cpu and empty_disk:

        # check the confidence interval every 20 cycles
        if (regens - 1) % 20 == 0 and regens != 1:

            # calculate s ^ 2
            y_bar = average(sum_response_times)
            c_bar = average(jobs_per_cycle)
            n = regens - 1

            sy = 0
            sc = 0
            syc = 0
            for i in range(1,regens):
                sy += (sum_response_times[i] - y_bar) ** 2
                sc += (jobs_per_cycle[i] - c_bar) ** 2
                syc += (sum_response_times[i] - y_bar) * (jobs_per_cycle[i] - c_bar)

            sy /= n - 1
            sc /= n - 1
            syc /= n - 1
            R = average(sum_response_times) / average(jobs_per_cycle)
            s = sy - 2 * R * syc + R ** 2 * sc
            
            # z for 0.95
            z1_a_2 =  1.960
            confidence_interval = z1_a_2 * float(np.sqrt(s)) / (c_bar * float(np.sqrt(n)))

            if confidence_interval / R < 0.1:
                Checkpoint = False
                break

        # register cycle length
        cycle_length = clock-old_clock
        cycle_lengths.append(clock-old_clock)
        old_clock = clock
       
        
        R_cycle = 0
        if jobs:
            for job in jobs:
                R_cycle += (departure_time[job] - arrival_time[job]+234)

            sum_response_times.append(R_cycle)
            jobs_per_cycle.append(len(jobs))

            
        else:
            sum_response_times.append(0)
            jobs_per_cycle.append(0)


        
        if cycle_length == 0:
            cycle_length = 1

 
        
        empty_cpu_times.append(cpu_EMPTY_TIME)
        cpu_EMPTY_TIME = 0
    

        
        
        empty_disk_times.append(disk_EMPTY_TIME)
        disk_EMPTY_TIME = 0
        

        regens += 1
        clock = cpu_arrival
        cpu_EMPTY_TIME += (clock - cpu_LAST_EMPTY)

        jobs = []

        # begin the job's execution

        processing_time = arrivals(54)
        cpu_current_job = job_id
        jobs.append(job_id)
        cpu_start_time = clock
        cpu_remaining_time = processing_time

        # define new arrival time and increment number of jobs
        arrival_time[job_id] = clock
        disk_visits[job_id] = 0
        cpu_delay[job_id] = 0
        cpu_arrival = clock + nextArrival()
        job_id += 1
        curr_jobs += 1
        empty_cpu = False
      

    elif empty_disk:

        # handle new arrival
        if next(cpu_arrival, cpu_start_time + cpu_remaining_time,None) == "arrival":
            clock = cpu_arrival
            arrival_time[job_id] = clock
            disk_visits[job_id] = 0
            cpu_delay[job_id] = 0
            jobs.append(job_id)
            processing_time = arrivals(54)
            cpu_queue.append([job_id,processing_time,clock])
            # define new arrival time and increment number of jobs
            cpu_arrival = clock + nextArrival()
            empty_cpu = False
            curr_jobs += 1
            job_id += 1
           

        else:
            clock = cpu_start_time + cpu_remaining_time
            # job leaves the system with p = 1/19 
            Uniform = U()
            if (Uniform <= 1/19):
                departure_time[cpu_current_job] = clock
                if cpu_queue:
                    max = -1
                    index = 0
                    max_index = 0
                    arr_time = float('inf')
                    for [jb_id,_,last_time] in cpu_queue:
                      if(clock-last_time+cpu_delay[jb_id] > max):
                        max = clock-last_time+cpu_delay[jb_id] 
                        max_index = index
                        arr_time = arrival_time[jb_id]
                      elif (clock-last_time+cpu_delay[jb_id] == max):
                        if(arrival_time[jb_id] < arr_time):
                          max_index = index
                          max = clock-last_time+cpu_delay[jb_id] 
                          arr_time = arrival_time[jb_id]
                      index = index + 1
                    
                    
                    cpu_current_job, cpu_remaining_time, last_time = cpu_queue.pop(max_index)
                    cpu_delay[cpu_current_job] += clock - last_time
                    cpu_start_time = clock
                   
                elif not cpu_queue:
                    cpu_current_job,  cpu_remaining_time, cpu_start_time = None, None,  None
                    cpu_LAST_EMPTY = clock
                    empty_cpu = True
                curr_jobs -= 1
            else:
                # load job to the empty disk
                disk_visits[cpu_current_job] += 1
                processing_time = arrivals(35)
                disk_current_job = cpu_current_job
                disk_remaining_time = processing_time
                disk_start_time = clock
                disk_EMPTY_TIME += (clock - disk_LAST_EMPTY)
                empty_disk = False

                # load new job to the cpu
                if cpu_queue:
                    max = -1
                    index = 0
                    max_index = 0
                    arr_time = float('inf')
                    for [jb_id,_,last_time] in cpu_queue:
                      if(clock-last_time+cpu_delay[jb_id] > max):
                        max = clock-last_time+cpu_delay[jb_id] 
                        max_index = index
                        arr_time = arrival_time[jb_id]
                      elif (clock-last_time+cpu_delay[jb_id] == max):
                        if(arrival_time[jb_id] < arr_time):
                          max_index = index
                          max = clock-last_time+cpu_delay[jb_id] 
                          arr_time = arrival_time[jb_id]
                      index = index + 1
                    
                    
                    cpu_current_job, cpu_remaining_time, last_time = cpu_queue.pop(max_index)
                    cpu_delay[cpu_current_job] += clock - last_time
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job, cpu_remaining_time, cpu_start_time,  = None,  None, None
                    empty_cpu = True
           


    elif empty_cpu:

        # handle new arrival
        if next(cpu_arrival, None, disk_start_time + disk_remaining_time) == "arrival":
            clock = cpu_arrival
            arrival_time[job_id] = clock
            disk_visits[job_id] = 0
            cpu_delay[job_id] = 0
            jobs.append(job_id)
            cpu_remaining_time = arrivals(54)
            cpu_EMPTY_TIME += (clock - cpu_LAST_EMPTY)
            cpu_current_job = job_id
            cpu_start_time = clock
            cpu_arrival = clock + nextArrival()
            empty_cpu = False
            curr_jobs += 1
            job_id += 1
            

        else:
            clock = disk_start_time + disk_remaining_time
           # load job to the empty cpu
            processing_time = arrivals(54)
            cpu_current_job = disk_current_job 
            cpu_remaining_time = processing_time
            cpu_start_time = clock
            empty_cpu = False

            # load new job to the disk
            if disk_queue:
                disk_current_job, disk_remaining_time = disk_queue.popleft()
                disk_start_time = clock
            elif not disk_queue:
                disk_current_job, disk_remaining_time, disk_start_time = None,None, None
                disk_LAST_EMPTY = clock
                empty_disk = True
            

    else:

        # handle new arrival
        if next(cpu_arrival, cpu_start_time + cpu_remaining_time, disk_start_time + disk_remaining_time) == "arrival":
            clock = cpu_arrival
            arrival_time[job_id] = clock
            disk_visits[job_id] = 0
            jobs.append(job_id)
            cpu_delay[job_id] = 0
            processing_time = arrivals(54)
            cpu_queue.append([job_id,processing_time,clock])
            # define new arrival time and increment number of jobs
            cpu_arrival = clock + nextArrival()
            empty_cpu = False
            curr_jobs += 1
            job_id += 1
            

        elif next(cpu_arrival, cpu_start_time + cpu_remaining_time, disk_start_time + disk_remaining_time) == "CPU":
            clock = cpu_start_time + cpu_remaining_time
            # job leaves the system with p = 1/19
            Uniform = U()
            if (Uniform <= 1/19):
                departure_time[cpu_current_job] = clock
                if cpu_queue:
                    max = -1
                    index = 0
                    max_index = 0
                    arr_time = float('inf')
                    for [jb_id,_,last_time] in cpu_queue:
                      if(clock-last_time+cpu_delay[jb_id] > max):
                        max = clock-last_time+cpu_delay[jb_id] 
                        max_index = index
                        arr_time = arrival_time[jb_id]
                      elif (clock-last_time+cpu_delay[jb_id] == max):
                        if(arrival_time[jb_id] < arr_time):
                          max_index = index
                          max = clock-last_time+cpu_delay[jb_id] 
                          arr_time = arrival_time[jb_id]
                      index = index + 1
                    
                    
                    cpu_current_job, cpu_remaining_time, last_time = cpu_queue.pop(max_index)
                    cpu_delay[cpu_current_job] += clock - last_time
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job, cpu_remaining_time, cpu_start_time = None, None, None
                    cpu_LAST_EMPTY = clock
                    empty_cpu = True
                curr_jobs -= 1
            else:
                # load job to the disk queue
                disk_visits[cpu_current_job] += 1
                processing_time = arrivals(35)
            
                disk_queue.append([cpu_current_job,processing_time])

                # load new job to the cpu
                if cpu_queue:
                    max = -1
                    index = 0
                    max_index = 0
                    arr_time = float('inf')
                    for [jb_id,_,last_time] in cpu_queue:
                      if(clock-last_time+cpu_delay[jb_id] > max):
                        max = clock-last_time+cpu_delay[jb_id] 
                        max_index = index
                        arr_time = arrival_time[jb_id]
                      elif (clock-last_time+cpu_delay[jb_id] == max):
                        if(arrival_time[jb_id] < arr_time):
                          max_index = index
                          max = clock-last_time+cpu_delay[jb_id] 
                          arr_time = arrival_time[jb_id]
                      index = index + 1
                    
                    
                    cpu_current_job, cpu_remaining_time, last_time = cpu_queue.pop(max_index)
                    cpu_delay[cpu_current_job] += clock - last_time
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job, cpu_remaining_time, cpu_start_time = None,None, None
                    cpu_LAST_EMPTY = clock
                    empty_cpu = True
            
        else:
            clock = disk_start_time + disk_remaining_time
            # load job to the cpu
            processing_time = arrivals(54)
            cpu_queue.append([disk_current_job,processing_time,clock])
         

            # load new job to the disk
            if disk_queue:
                disk_current_job, disk_remaining_time = disk_queue.popleft()
                disk_start_time = clock
            elif not disk_queue:
                disk_current_job, disk_remaining_time, disk_start_time = None, None, None
                disk_LAST_EMPTY = clock
                empty_disk = True
            

print("Regen cycles =",regens)



print("Average response time ",round(float(average(sum_response_times)/average(jobs_per_cycle)),3))
print("cpu utilization",round(float(1 - average(empty_cpu_times)/average(cycle_lengths)),3))
print("Disk utilization",round(float(1 - average(empty_disk_times)/average(cycle_lengths)),3))




Regen cycles = 1000
Average response time  13748.357
cpu utilization 0.94
Disk utilization 0.564


## Lruf

In [19]:


#cpu_last_time -> otan mpaineis stin cpu (oxi stin oura)
#cpu_delay -> otan kati vgainei apo cpu kanoume clock-cpu_last_time
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

def average(list_A):
    return(sum(list_A)/(len(list_A)))

#for both poisson arrivals and exponential distributed times of service we use the generator -logU/a
#for poissoin arrivals we have a 0.9 arrivals/1000msec -> -logU*(1000/0.9) msec
#for exponential distributed times of service we have 1/a = 54msec for cpu and 1/a = 35msec
def arrivals(average_time,num_points = 1):
    X = - float(np.log(U(num_points))) * average_time
    moment_of_arrivals[average_time].append(X)
    return X

def nextArrival():
    # we have a = 1000/0.9 msec (we use 1000 in order to measure time in mseconds)
    average_time = 1000/0.9
    return arrivals(average_time)

def U(num_points = 1):
    return float(np.random.uniform(size = num_points))


def next(arrival,cpu,disk):
    if cpu == None:
        if disk != None and disk < arrival:
            return "DISK"
        else:
            return "arrival"
    elif disk == None:
        if cpu != None and cpu < arrival:
            return "CPU"
        else:
            return "arrival"
    else:
        if arrival < disk  and arrival < cpu:
            return "arrival"
        elif cpu < disk:
            return "CPU"
        else:
            return "DISK"






# Initialize

cpu_queue = deque([])
disk_queue = deque([])

cpu_current_job = None
cpu_start_time = None
cpu_remaining_time = None

disk_current_job = None
disk_start_time = None
disk_remaining_time = None

job_id = 1
clock = 0
curr_jobs = 0
old_clock = 0

moment_of_arrivals = {}
for average_time in [1000/0.9, 54, 35]:
    moment_of_arrivals[average_time] = []

# Initialize statistics

arrival_time = {}
cpu_last_time = {}
departure_time = {}
disk_visits = {}
cpu_delay = {}


sum_response_times = []



empty_cpu_times = []
empty_disk_times = []

cycle_lengths   = []
jobs_per_cycle = []


cpu_EMPTY_TIME  = 0
disk_EMPTY_TIME = 0


cpu_arrival = nextArrival()
disk_EMPTY_TIME = cpu_EMPTY_TIME = 0
disk_LAST_EMPTY = cpu_LAST_EMPTY = 0


empty_cpu = True



empty_disk = True
regens = 0


jobs = []

Checkpoint = True

while (Checkpoint and regens < 1000):



    # start of the regen cycle
    # Everything is empty
    if empty_cpu and empty_disk:

        # check the confidence interval every 20 cycles
        if (regens - 1) % 20 == 0 and regens != 1:

            # calculate s ^ 2
            y_bar = average(sum_response_times)
            c_bar = average(jobs_per_cycle)
            n = regens - 1

            sy = 0
            sc = 0
            syc = 0
            for i in range(1,regens):
                sy += (sum_response_times[i] - y_bar) ** 2
                sc += (jobs_per_cycle[i] - c_bar) ** 2
                syc += (sum_response_times[i] - y_bar) * (jobs_per_cycle[i] - c_bar)

            sy /= n - 1
            sc /= n - 1
            syc /= n - 1
            R = average(sum_response_times) / average(jobs_per_cycle)
            s = sy - 2 * R * syc + R ** 2 * sc
            
            # z for 0.95
            z1_a_2 =  1.960
            confidence_interval = z1_a_2 * float(np.sqrt(s)) / (c_bar * float(np.sqrt(n)))

            if confidence_interval / R < 0.1:
                Checkpoint = False
                break

        # cycle length
        cycle_length = clock-old_clock
        cycle_lengths.append(clock-old_clock)
        old_clock = clock
       
    
        R_cycle = 0
        if jobs:
            for job in jobs:
                R_cycle += (departure_time[job] - arrival_time[job]+234)
 
            sum_response_times.append(R_cycle)
            jobs_per_cycle.append(len(jobs))

           
        else:
            sum_response_times.append(0)
            jobs_per_cycle.append(0)


        
        if cycle_length == 0:
            cycle_length = 1

        
       
        empty_cpu_times.append(cpu_EMPTY_TIME)
        cpu_EMPTY_TIME = 0
       

        
        
        empty_disk_times.append(disk_EMPTY_TIME)
        disk_EMPTY_TIME = 0
      

        regens += 1
        clock = cpu_arrival
        cpu_EMPTY_TIME += (clock - cpu_LAST_EMPTY)

        jobs = []

        # begin the job's execution

        processing_time = arrivals(54)
        cpu_current_job = job_id
        jobs.append(job_id)
        cpu_start_time = clock
        cpu_remaining_time = processing_time
        cpu_last_time[job_id] = clock

        # define new arrival time and increment number of jobs
        arrival_time[job_id] = clock
        disk_visits[job_id] = 0
        cpu_delay[job_id] = 0
        cpu_arrival = clock + nextArrival()
        job_id += 1
        curr_jobs += 1
        empty_cpu = False
       

    elif empty_disk:

        #new arrival
        if next(cpu_arrival, cpu_start_time + cpu_remaining_time,None) == "arrival":
            clock = cpu_arrival
            arrival_time[job_id] = clock
            disk_visits[job_id] = 0
            cpu_delay[job_id] = 0
            jobs.append(job_id)
            processing_time = arrivals(54)
            cpu_queue.append([job_id,processing_time])
            # new arrival time 
            cpu_arrival = clock + nextArrival()
            empty_cpu = False
            curr_jobs += 1
            job_id += 1
           

        else:
            clock = cpu_start_time + cpu_remaining_time
            # job leaves the system with p = 1/19
            Uniform = U()
            if (Uniform <= 1/19):
                departure_time[cpu_current_job] = clock
                cpu_delay[cpu_current_job] = clock - cpu_last_time[cpu_current_job]
                if cpu_queue:
                    min = float('inf')
                    arr_time = float('inf')
                    index = 0
                    min_index = 0
                    for [jb_id,_] in cpu_queue:
                      if (cpu_delay[jb_id] < min):
                        min = cpu_delay[jb_id]
                        min_index = index
                        arr_time = arrival_time[jb_id]
                      elif (cpu_delay[jb_id] == min):
                        if(arrival_time[jb_id]< arr_time):
                          min = cpu_delay[jb_id]
                          min_index = index
                          arr_time = arrival_time[jb_id] 
                      index = index + 1 
                    
                    
                    cpu_current_job, cpu_remaining_time = cpu_queue[min_index]
                    cpu_last_time[cpu_current_job] = clock
                    del cpu_queue[min_index]

                    
                    cpu_start_time = clock
                   
                elif not cpu_queue:
                    cpu_current_job,  cpu_remaining_time, cpu_start_time= None, None,  None
                    cpu_LAST_EMPTY = clock
                    empty_cpu = True
                curr_jobs -= 1
            else:
                # load job to the empty disk
                disk_visits[cpu_current_job] += 1
                processing_time = arrivals(35)
                disk_current_job = cpu_current_job
                disk_remaining_time = processing_time
                disk_start_time = clock
                disk_EMPTY_TIME += (clock - disk_LAST_EMPTY)
                empty_disk = False

                # load new job to the cpu
                cpu_delay[cpu_current_job] = clock - cpu_last_time[cpu_current_job]
                if cpu_queue:
                    min = float('inf')
                    arr_time = float('inf')
                    index = 0
                    min_index = 0
                    for [jb_id,_] in cpu_queue:
                      if (cpu_delay[jb_id] < min):
                        min = cpu_delay[jb_id]
                        min_index = index
                        arr_time = arrival_time[jb_id]
                      elif (cpu_delay[jb_id] == min):
                        if(arrival_time[jb_id]< arr_time):
                          min = cpu_delay[jb_id]
                          min_index = index
                          arr_time = arrival_time[jb_id] 
                      index = index + 1 
                    
                    
                    cpu_current_job, cpu_remaining_time = cpu_queue[min_index]
                    cpu_last_time[cpu_current_job] = clock
                    del cpu_queue[min_index]

                    
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job, cpu_remaining_time, cpu_start_time = None,  None, None
                    empty_cpu = True
            


    elif empty_cpu:

        #new arrival
        if next(cpu_arrival, None, disk_start_time + disk_remaining_time) == "arrival":
            clock = cpu_arrival
            arrival_time[job_id] = clock
            disk_visits[job_id] = 0
            cpu_delay[job_id] = 0
            cpu_last_time[job_id] = clock
            jobs.append(job_id)
            cpu_remaining_time = arrivals(54)
            cpu_EMPTY_TIME += (clock - cpu_LAST_EMPTY)
            cpu_current_job = job_id
            cpu_start_time = clock
            cpu_arrival = clock + nextArrival()
            empty_cpu = False
            curr_jobs += 1
            job_id += 1
          

        else:
            clock = disk_start_time + disk_remaining_time
           # load job to the empty cpu
            processing_time = arrivals(54)
            cpu_current_job = disk_current_job 
            cpu_remaining_time = processing_time
            cpu_start_time = clock
            empty_cpu = False

            cpu_last_time[cpu_current_job] = clock


            # load new job to the disk
            if disk_queue:
                disk_current_job, disk_remaining_time = disk_queue.popleft()
                disk_start_time = clock
            elif not disk_queue:
                disk_current_job, disk_remaining_time, disk_start_time = None,None, None
                disk_LAST_EMPTY = clock
                empty_disk = True
           

    else:

        #new arrival
        if next(cpu_arrival, cpu_start_time + cpu_remaining_time, disk_start_time + disk_remaining_time) == "arrival":
            clock = cpu_arrival
            arrival_time[job_id] = clock
            disk_visits[job_id] = 0
            jobs.append(job_id)
            cpu_delay[job_id] = 0
            processing_time = arrivals(54)
            cpu_queue.append([job_id,processing_time])
            #new arrival time 
            cpu_arrival = clock + nextArrival()
            empty_cpu = False
            curr_jobs += 1
            job_id += 1
            

        elif next(cpu_arrival, cpu_start_time + cpu_remaining_time, disk_start_time + disk_remaining_time) == "CPU":
            clock = cpu_start_time + cpu_remaining_time
            # job leaves the system with p = 1/19 
            Uniform = U()
            if (Uniform <= 1/19):
                departure_time[cpu_current_job] = clock
                cpu_delay[cpu_current_job] = clock - cpu_last_time[cpu_current_job]
                if cpu_queue:
                    min = float('inf')
                    arr_time = float('inf')
                    index = 0
                    min_index = 0
                    for [jb_id,_] in cpu_queue:
                      if (cpu_delay[jb_id] < min):
                        min = cpu_delay[jb_id]
                        min_index = index
                        arr_time = arrival_time[jb_id]
                      elif (cpu_delay[jb_id] == min):
                        if(arrival_time[jb_id]< arr_time):
                          min = cpu_delay[jb_id]
                          min_index = index
                          arr_time = arrival_time[jb_id] 
                      index = index + 1 
                    
                    
                    cpu_current_job, cpu_remaining_time = cpu_queue[min_index]
                    cpu_last_time[cpu_current_job] = clock
                    del cpu_queue[min_index]

                    
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job, cpu_remaining_time, cpu_start_time = None, None, None
                    cpu_LAST_EMPTY = clock
                    empty_cpu = True
                curr_jobs -= 1
            else:
                # load job to the disk queue
                disk_visits[cpu_current_job] += 1
                processing_time = arrivals(35)
            
                disk_queue.append([cpu_current_job,processing_time])
                
               
                cpu_delay[cpu_current_job] = clock - cpu_last_time[cpu_current_job]
                if cpu_queue:
                    min = float('inf')
                    arr_time = float('inf')
                    index = 0
                    min_index = 0
                    for [jb_id,_] in cpu_queue:
                      if (cpu_delay[jb_id] < min):
                        min = cpu_delay[jb_id]
                        min_index = index
                        arr_time = arrival_time[jb_id]
                      elif (cpu_delay[jb_id] == min):
                        if(arrival_time[jb_id]< arr_time):
                          min = cpu_delay[jb_id]
                          min_index = index
                          arr_time = arrival_time[jb_id] 
                      index = index + 1 
                    
                    
                    cpu_current_job, cpu_remaining_time = cpu_queue[min_index]
                    cpu_last_time[cpu_current_job] = clock
                    del cpu_queue[min_index]

                    
                    cpu_start_time = clock
                elif not cpu_queue:
                    cpu_current_job, cpu_remaining_time, cpu_start_time = None,None, None
                    cpu_LAST_EMPTY = clock
                    empty_cpu = True
            

        else:
            clock = disk_start_time + disk_remaining_time
            # load job to the cpu
            processing_time = arrivals(54)
            # load job from the disk to the cpu
            cpu_queue.append([disk_current_job,processing_time])


            # load new job to the disk
            if disk_queue:
                disk_current_job, disk_remaining_time = disk_queue.popleft()
                disk_start_time = clock
            elif not disk_queue:
                disk_current_job, disk_remaining_time, disk_start_time = None, None, None
                disk_LAST_EMPTY = clock
                empty_disk = True
           

print("Regen cycles =",regens)




print("Average response time ",round(float(average(sum_response_times)/average(jobs_per_cycle)),3))
print("cpu utilization",round(float(1 - average(empty_cpu_times)/average(cycle_lengths)),3))
print("Disk utilization",round(float(1 - average(empty_disk_times)/average(cycle_lengths)),3))



Regen cycles = 1000
Average response time  14021.61
cpu utilization 0.945
Disk utilization 0.566
